In [1]:
import numpy as np

In [2]:
n = 100_000
d = 3

mu = np.array([1, 10, 20]).reshape(-1, 1)
Sigma = np.array([1, -1, 0, -1, 3, -1, 0, -1, 3]).reshape(3, 3)

X = np.random.multivariate_normal(mean=mu.flatten(), cov=Sigma, size=n)

wt = 1  # Can modify this to take an exponentially weighted moving average
wtSum = 0.0
muhat = np.zeros((3, 1))

# Starting estimates of covariance / inverse covariance.
M = 0 * np.eye(d)
Minv = 100_000_000 * np.eye(d)

for x in X:
    x = x.reshape(-1, 1)
    wtSum = wt * wtSum + 1  # Divisor For Covariance Matrix
    delta = x - muhat
    muhat += delta / wtSum
    new_delta = (x - muhat).reshape(1, -1)

    # Estimation of Sigma
    M = wt * M + delta @ new_delta
    Sigma_hat = M / wtSum

    # Estimation of the inverse of Sigma
    Mnum = 1.0 / np.power(wt, 2) * Minv @ delta @ new_delta @ Minv
    Mden = 1.0 + 1.0 / wt * new_delta @ Minv @ delta
    Minv = Minv / wt - Mnum / Mden.item()

print("Error between estimates (should be very small): ")
print(np.linalg.pinv(Sigma_hat) - Minv * wtSum)

print("Error between inverse of real and estimated covariance matrix:")
print(np.linalg.pinv(Sigma) - Minv * wtSum)

Error between estimates (should be very small): 
[[ 7.16982029e-13 -1.06115117e-12  8.38745740e-13]
 [ 1.27098332e-12 -1.06858966e-12  5.70127279e-13]
 [-7.64749375e-13  1.02587383e-12 -4.65960603e-13]]
Error between inverse of real and estimated covariance matrix:
[[0.0025169  0.00425884 0.00152781]
 [0.00425884 0.00275907 0.00158127]
 [0.00152781 0.00158127 0.00022925]]
